# 自動走行(カメラなし)

ここでは、TensorRTに変換されたモデルを読み込み、自動走行にチャレンジしてみます。

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('model_trt.pth'))

FileNotFoundError: [Errno 2] No such file or directory: 'model_trt.pth'

racecarクラスのインスタンスの作成

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

cameraクラスのインスタンスの作成

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=60)

最後に、以下のセルを実行すると、頂点のx値に基づいてレースカーを操り、レースカーを前進させることができます。

以下にヒントを示します。

* 車が左右にふらつく場合は、ステアリングゲインを下げてください。
* 車が曲がりきれない場合は、ステアリングゲインを上げる。
* 車が右に傾いている場合は、ステアリングバイアスをよりネガティブにする（-0.05のような小さな単位で）。
* 車が左に傾いたら、ステアリング・バイアスをよりポジティブにする（小刻みに +0.05）。


In [ ]:
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets

network_output_slider = widgets.FloatSlider(description='推論結果', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='ゲイン', min=-1.0, max=1.0, value=-0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='バイアス', min=-0.5, max=0.5, value=-0.1, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='ハンドル値', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='速度(正:前, 負:後)', min=-1.0, max=1.0, value=0, step=0.01, orientation='vertical')


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
#throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([network_output_slider,
                       widgets.Label(value="X"),
                       steering_gain_slider,
                       widgets.Label(value="+"),
                       steering_bias_slider,
                       widgets.Label(value="||"), 
                       steering_value_slider], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         throttle_slider]
    )
)

# 実行

下記セルを実行すると、推論結果から、上記に実行されているWidgetのハンドル角度が計算されはじめます。速度のゲージは下記セルのプログラムを実行した祭の初期値が反映されます。

下記プログラムを停止するには■ボタンをクリックし、停止します。

In [ ]:
from utils import preprocess
import numpy as np

car.throttle = -1.0 * throttle_slider.value - 0.1 ## TT02用に修正

try:
    while True:
        throttle_slider.disabled = True
        steering_gain_slider.disabled = True
        steering_bias_slider.disabled = True
    
        image = camera.read()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
    
        car.steering = x * steering_gain_slider.value + steering_bias_slider.value
        steering_value_slider.value = -car.steering
    
except KeyboardInterrupt:
    throttle_slider.disabled = False
    steering_gain_slider.disabled = False
    steering_bias_slider.disabled = False

# カメラの終了処理(必須)
再度、データセット追加や学習は、'02_inreragtive_regression.ipynb'に戻りおこないます。
その際に、下記セルを実行して、カメラの終了処理を忘れずにおこなってください。

In [1]:
import time
camera.running = False
time.sleep(1)
camera.cap.release()

NameError: name 'camera' is not defined